In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [3]:
from torchModel import NextTokenLSTM

In [7]:
model = NextTokenLSTM(len(token_to_int)).to(device)
model.load_state_dict(torch.load("models/wordLSTM.pth", weights_only=True))
print(model)

NextTokenLSTM(
  (embedding): Embedding(14094, 128)
  (lstm): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.2)
  (full_connected): Linear(in_features=128, out_features=14094, bias=True)
)


In [4]:
import pickle

In [5]:
with open("data/token_to_int.pkl", "rb") as file:
    token_to_int=pickle.load(file)

with open("data/int_to_token.pkl", "rb") as file:
    int_to_token=pickle.load(file)

## Simple sampling by result

In [8]:
import numpy as np

In [10]:
SEQ_LEN = 100 # how many time step for RNN

In [14]:
def sample(model, sen_start, max_len=200):
    model.eval()
    text = sen_start.lower().split(' ')
    hc = model.initalize_hidden(1)
    length = max_len - len(text)
    for _ in range(0, length):
        if len(text) <= SEQ_LEN:
            x = torch.tensor([[token_to_int[w] for w in text]])
        else:
            x = torch.tensor([[token_to_int[w] for w in text[-SEQ_LEN:]]])
        inputs = x.to(device)
        outputs, hc = model(inputs, hc)
        logits = outputs[0][-1]
        prob = nn.functional.softmax(logits, dim=0).detach().cpu().numpy()
        idx = np.random.choice(len(logits), p=prob)
        text.append(int_to_token[idx])
    sentence = " ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        sentence = sentence.replace(f" {m}", f"{m} ")
    sentence = sentence.replace('"  ', '"')
    sentence = sentence.replace("'  ", "'")
    sentence = sentence.replace('" ', '"')
    sentence = sentence.replace("' ", "'")
    return sentence
    

In [17]:
torch.manual_seed(42)
np.random.seed(42)
print(sample(model, sen_start='Anna and the prince'))  

anna and the prince had explained to him,  in the habitual world with which they would not be made in what a new secretary,  and he began so much he saw the bell.  the culprit mihail clerk had been lighted and too half the acre,  he was interrupted.  "sappho says seryozha too very exquisite,  "he said to levin.  "i don't know advice about you,  "said sergey ivanovitch,  smiling.  he stood about,  and he walked away and saw.  one past later.  ""no,  that's it?  will you see or not for you?  why?  it's a foolish time my weakness but a man in nature very.  it's a wonder before me my heart,  completely exalted,  but not i'm taking a creature coming to more family yesterday.  i told you about his book,  you have refused to do,  and you're not pleased in being any points,  and i didn't want to take their conversation,  then we shall have a


In [19]:
torch.manual_seed(52)
np.random.seed(52)
print(sample(model, sen_start='Anna and the prince'))  

anna and the prince should not be.  "can the widow knife to bed again themselves.  those young landowners, 've been in a bad german,  "vronsky seemed to take an acquaintance as he sat down in silence,  looking at his elbows with anna,  "then there's no hurry for!  what is it?  "(  anna expressed an answer that the children would have heard balls the day in a shadow.  that day a straw sight of it.  "what is it?  thank starting?  ""that business didn't let or be so much.  "vronsky was standing at getting up to stop and to stop herself.  "how do you care?  "she said,  "i believe it's selfishness on her tiny little coat.  that's certain.  come,  please,  sir,  "said alexey alexandrovitch,  getting up.  giving him several seconds he tried to find the pleasure when everything was allowed to sound,  the kitchen,  some one left up the staircase,  the feeling that had a


## Simple sampling by top-k 

In [22]:
def topk_sample(model, sen_start, top_k, temperature=1, max_len=200):
    model.eval()
    text = sen_start.lower().split(' ')
    hc = model.initalize_hidden(1)
    length = max_len - len(text)
    for _ in range(0, length):
        if len(text) <= SEQ_LEN:
            x = torch.tensor([[token_to_int[w] for w in text]])
        else:
            x = torch.tensor([[token_to_int[w] for w in text[-SEQ_LEN:]]])
        inputs = x.to(device)
        outputs, hc = model(inputs, hc)
        logits = outputs[0][-1]
        logits = logits/temperature
        #prob = nn.functional.softmax(logits, dim=0).detach().cpu().numpy()
        prob = nn.functional.softmax(logits, dim=0).detach().cpu()
        ps, tops = prob.topk(top_k)
        ps = ps/ps.sum()
        idx = np.random.choice(tops, p=ps.numpy())
        text.append(int_to_token[idx])
    sentence = " ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        sentence = sentence.replace(f" {m}", f"{m} ")
    sentence = sentence.replace('"  ', '"')
    sentence = sentence.replace("'  ", "'")
    sentence = sentence.replace('" ', '"')
    sentence = sentence.replace("' ", "'")
    return sentence

In [23]:
torch.manual_seed(42)
np.random.seed(42)
print(topk_sample(model, sen_start='Anna and the prince', top_k=3,temperature=0.5 )) 

anna and the prince had to leave her husband.  she knew that his mother was not married,  and would have liked to be married.  she was at ease with him.  she knew that her winter,  her father,  who had been going in the winter,  she was not in returning on his mind.  she was sitting in the middle of the room,  and the nurse came in her eyes,  and she saw that she was ashamed of.  she was glad of it,  and could not admit that in her soul she could not help following her.  the tears and happiness in her eyes,  revived it unseen expression of brilliance,  cruel aversion,  prevented him,  and squeezing his hand,  and with the same despondent face gazed at her.  vronsky,  with a long overcoat,  gazed at the looking-glass,  and then with a smile of her eyes,  made something thin,  and was appalled at her movements.  she was not calm,  but was embarrassed with her.  the smile was drawn by her childish eyes.  she saw the sound


In [24]:
torch.manual_seed(52)
np.random.seed(52)
print(topk_sample(model, sen_start='Anna and the prince', top_k=3,temperature=0.5 )) 

anna and the prince had explained to her.  but he was a feeling that he was not to blame.  he was not to blame for this instant,  and he had been a long while before,  though he was unmanly and effeminate,  effeminate,  and lack of wealthy significance.  "you've been so fond of your sister,  "said stepan arkadyevitch,  smiling.  "i'm going to the bee house,  i'll send you a shirt.  ""i'm not afraid you've been here for me.  ""well,  then,  i'm not a negro. . .  ""oh,  well,  then,  "he said to himself,  and went into the room.  "i don't want to go to bryansky,  and i'll go and see him.  ""oh,  i'm going on,  "said the englishman.  "i've come to fetch you.  ""well,  what do you say to pyramids?  ""oh,  i
